In [1]:
#Marsha Gomez Gomez
#MIRCV 2021
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install whoosh

     |████████████████████████████████| 471kB 5.5MB/s 


In [3]:
# %matplotlib inline

from whoosh import index
from whoosh.fields import *
from whoosh.analysis import *
from whoosh import qparser
import os, os.path

from matplotlib import pyplot as plt
from IPython.display import display, HTML

from base64 import b64encode
from urllib.parse import quote
from io import BytesIO

BASE_DIR = '/content/gdrive/My Drive/mircv2021'

COCO_CAPTIONS_DATASET = BASE_DIR+'/data/txt/captions.properties'
COCO_ANNOTATIONS_DATASET = BASE_DIR+'/data/txt/annotations.properties'
INDEX_NAME = 'coco'
IMG_DIR = BASE_DIR+'/data/coco_img'

print('Setting Environment Variables. Done')

Setting Environment Variables. Done


In [4]:
#TODO Indexing

if not os.path.exists(BASE_DIR + "/out"):
    os.mkdir(BASE_DIR + "/out")
if not os.path.exists(BASE_DIR + "/out/whoosh"):
    os.mkdir(BASE_DIR + "/out/whoosh")

print('creating ' + INDEX_NAME + ' index...')

schema = Schema(id=ID(stored=True),
                text=TEXT(analyzer=KeywordAnalyzer(),stored=True),
                annotation=TEXT(analyzer=KeywordAnalyzer(),stored=True))
#create whoosh index
ix = index.create_in(BASE_DIR + "/out/whoosh", schema)

print('indexing done ')


creating coco index...
indexing done 


In [6]:
class DNNExtractor:    
    
    def __init__(self, net_proto_path, trained_model_path, size, mean_values=None):
        self.size = size
        self.mean_values = mean_values
        #TODO
        #Load Caffe Weights
        self.net = cv2.dnn.readNetFromCaffe(net_proto_path, trained_model_path)
        
        # to enable GPU (this won't work on Colab without recompiling opencv)
        #self.net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
        #self.net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
    
    def extract(self, img_file, layer, normalize=False):
        #TODO
        #Load img_file and extract fetures calling DNN 
        img = cv2.imread(img_file)
        blob = cv2.dnn.blobFromImage(img, 1.0, self.size, self.mean_values, swapRB=False, crop=False)
        self.net.setInput(blob)
        features = self.net.forward(layer).flatten()

        #normalize features if normalize = True
        if normalize == True:
          features = features/np.linalg.norm(features)
        #return the features
        return features
    
# it creates an instance of the DNNExtractor class
dnn = DNNExtractor(DEEP_PROTO, DEEP_MODEL, SIZE, MEAN_VALUES)
print("Done DNN")

NameError: ignored

In [ ]:
def get_table(img, title, txt):
    table ='<table><td><img style="width:320px" src="' + img + '"></td>'
    table +='<td style="text-align:left"><b>' + title + '</b></p>' + txt + '</td></table>'
    return table


def display_result(img_path, title, txt):
    with open(img_path, 'rb') as f:
        dataEncoded = b64encode(BytesIO(f.read()).getvalue())
        data = 'data:image/jpg;base64,{}'.format(quote(dataEncoded))
        display(HTML(get_table(data, title, txt)))

In [ ]:
#TODO Searching
#initialize whoosh searcher
searcher = ix.searcher()

#txt = "annotation:person text:people text:bike"
#txt = "annotation:person text:people bike"

#txt = "annotation:person text:people NOT text:bike"
#txt = "annotation:person text:beach OR text:bird"
txt = "annotation:person text:people text:bike"


print("query: " + txt)


#perform txt query
parser = qparser.QueryParser("text", ix.schema, group=qparser.AndGroup)
txt_query = parser.parse(txt)
res = searcher.search(txt_query) 

print(res[0])
print(res[0].score)



query: annotation:person text:people text:bike
<Hit {'annotation': 'vehicle motorcycle person person person person person person person person person person person person person person person person person person person person\n', 'id': '000000007816', 'text': 'A person on a motor bike travels around a sharp corner. A person on a motor bike on a road. a rider dressed in white riding a matching white motorcycle A person on a motorcycles driving past a group of people behind a fence. A guy on a motorcycle with some people watching.\n'}>
13.894059874484046


In [ ]:
for hit in res:
  score = hit.score 
  id = hit["id"]
  text = hit["text"]
  filename = id+".jpg"
  filepath = IMG_DIR+"/"+filename
  title = "ID: "+ id + " SCORE: "+ str(score)
  #print(id,":",text,'score: ', score)

#display the search results
display_result(filepath,title,text)
print("filename"+filepath)


filename/content/gdrive/My Drive/mircv2021/data/coco_img/000000395180.jpg
